In [1]:
from sentence_transformers import SentenceTransformer
import os
import re
import codecs
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from scipy.spatial.distance import cosine

%matplotlib inline

# !pip install transformers
# !pip install sentence_transformers

In [2]:
name_of_folder_for_processing = 'sampled_raw'
Country_to_process = 'IN'

In [3]:
base_dir = os.path.join(os.getcwd(), 'data', name_of_folder_for_processing) 
countries_list = os.listdir(base_dir) # dir is your directory path
summary = []

for con in countries_list:
    if(os.path.isdir(os.path.join(base_dir,con))):
        print(con, end=", ", flush=True)
        publishers = os.listdir(os.path.join(base_dir,con))
        for pub in publishers:
            subdir = os.path.join(base_dir, con, pub)
            length = sum([len(files) for r, d, files in os.walk(subdir)])
            summary.append([con, pub, length])

summary_df = pd.DataFrame(summary, columns = ['country','publisher','count'])     

PK, US, HK, GH, NG, KE, NZ, BD, IN, LK, CA, ZA, SG, PH, GB, MY, AU, IE, JM, TZ, 

In [17]:
summary_df[summary_df.country=='IN'].sort_values(by=['count'])

,country,publisher,count
1332,IN,sportscafe,29
1183,IN,ettech-com,29
1264,IN,ettelecom-com,30
1199,IN,investing-com,30
1250,IN,defence-aviation-post,30
...,...,...,...
1283,IN,livemint,3443
1359,IN,daily-news-analysis,3493
1253,IN,economic-times,4796
1231,IN,the-hindu,7776


In [5]:
# idx = summary_df.groupby(['country'])['count'].transform(max) == summary_df['count']
# max_table = summary_df[idx]
# max_table

In [6]:
overall_dict = {}

for index, row in summary_df.iterrows():
    country, publisher = row[0], row[1]
    if country != Country_to_process:
        pass
    else:
        
        print(country, end=", ", flush=True)
        directory = os.path.join(base_dir, country, publisher)
        df_dict = {"id": [], "title": [], "publisher": [], "article_text":[],"url":[], "path":[]}
        if not os.path.isdir(directory):
            pass
        else:
            for entry in os.scandir(directory):
                if(os.path.isdir(entry)):
                    for entry_2 in os.scandir(entry.path):
                        try:
                            with open(entry_2, "r") as f:
                                article_id = f.readline().strip()
                                article_title = f.readline().strip()
                                publisher = f.readline().strip()
                                url = f.readline()
                                f.readline()
                                article_text = f.readline().strip()
                                df_dict['url'].append(url)
                                df_dict['id'].append(article_id)
                                df_dict['title'].append(article_title)
                                df_dict['publisher'].append(publisher)
                                df_dict['article_text'].append(article_text)
                                df_dict['path'].append(entry_2)
                        except:
                            pass
            sample_df = pd.DataFrame.from_dict(df_dict)
            #sample_df = sample_df.sample(n=5, random_state=1)

            if country not in overall_dict.keys():
                overall_dict[country] = {}
            overall_dict[country][publisher] = sample_df


IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, IN, 

## Preprocessing


In [7]:
# import a clean function data
def clean_text(text):
    return text.strip()

def text_initial_preproecss(df):

    df = df.copy()
    df = df[~pd.to_numeric(df['id'], errors='coerce').isnull()]
    df["id"] = df["id"].astype(int)
    df['article_text'] = df.article_text.str.lower()
    df["article_text"] = df["article_text"].apply(clean_text)
    df['text'] = df['article_text']
    reports = df
    
    # Puncutation preprocesing
    reports['text'] = reports.text.str.replace('{', '')
    reports['text'] = reports.text.str.replace('}', '')
    reports['text'] = reports.text.str.replace("\n", '')
    reports['text'] = reports.text.str.rstrip("\n") #remove empty lines
    reports['text'] = reports.text.str.replace("@ @ @ @ @ @ @ @ @ @ ", '')
    reports['text'] = reports.text.str.replace(" @", '')
    reports['text'] = reports.text.str.replace(" '", "'")
    reports['text'] = reports.text.str.replace("\"", "")
    reports['text'] = reports.text.str.replace(",", "")
    reports['text'] = reports.text.str.replace("(", "")
    reports['text'] = reports.text.str.replace(")", "")
    reports['text'] = reports.text.str.replace(" <p>", ".")
    reports['text'] = reports.text.str.replace(" <h>", ".")
    reports['text'] = reports.text.str.replace("<p>", "")
    reports['text'] = reports.text.str.replace("<h>", "")
    reports['text'] = reports.text.str.replace('<', '')
    reports['text'] = reports.text.str.replace('>', '')
    reports['text'] = reports.text.str.replace(":", "")
    reports['text'] = reports.text.str.replace("?", ".")
    reports['text'] = reports.text.str.replace("!", ".")
    reports['text'] = reports.text.str.replace(r"\.\s[\.\s]+", ". ") #converting . . to .
    reports['text'] = reports.text.str.replace(r"\.+", ".") #converting ... to .
    reports['text'] = reports.text.str.replace("--", "") 
    reports['text'] = reports.text.str.replace("-", " ")
    reports['text'] = reports.text.str.replace(" +", " ")
    reports['text'] = reports.text.str.replace(" n't", "n't")
    
    return reports.copy()

In [8]:
import platform
os_name = platform.system()

def save_file(report,base):
#     print(report)
#     print(type(report))
    report = report.squeeze()
        
    full_path = report.publisher
    file_name = str(report.id) + ".txt"
    
    if os_name == 'Windows':
        
        base_dir = 'data\preprocessed\\{}'.format(base)
        con = report.path.path.split('{0}\\'.format(name_of_folder_for_processing))[1]
        
    else:
        
        base_dir = 'data/preprocessed/{}'.format(base)
        con = report.path.path.split('/{0}/'.format(name_of_folder_for_processing))[1]
    
    path = os.path.join(base_dir,con)
    
    full_path = os.path.dirname(path)
    file_name = os.path.basename(path)

    pathlib.Path(full_path).mkdir(parents=True, exist_ok=True)
    
    with codecs.open(f"{full_path}/{file_name}", "w", encoding = "utf-8") as f:
        id_number = str(report.id) if not pd.isna(report.id) else ""
        title = report.title if not pd.isna(report.title) else ""
        website = report.publisher if not pd.isna(report.publisher) else ""
        url = report.url if not pd.isna(report.url) else ""
        f.writelines([id_number, "\n",
                      title, "\n",
                      website, "\n",
                      url, "\n\n", report.text])
        f.close()

In [9]:
def ngram_preprocess(df):
    time_taken = []
    
#     reports_temp = df[df['publisher']==article][['id', 'text','text_vect']].copy()
    if len(df) > 500:
        reports_temp = df.sample(n=500, random_state=1).copy()
    else:
        reports_temp = df.copy()
        
    reports_temp['text_vect'] = reports_temp.text.apply(sent_tokenize)
#     display(reports_temp.head(3))
    word_vectorizer = CountVectorizer(ngram_range=(5,5), stop_words=[])
    flatten = [item for sublist in reports_temp['text_vect'] for item in sublist]
    sparse_matrix = word_vectorizer.fit_transform(flatten)

    frequency = sum(sparse_matrix).toarray()[0]
    frequency_df = pd.DataFrame(frequency, index=word_vectorizer.get_feature_names(),columns = ['frequency']).sort_values(by=['frequency'],ascending=False)

    freq_above_10 = frequency_df[frequency_df['frequency'] >= max((len(reports_temp)/10),3)] #len(MAXSIZE)/2
    phrase_list = list(freq_above_10.index)

    sentences_to_remove = []

    for phrase in phrase_list:
#         removing_sent = set([sent for sent in flatten if phrase in sent])
        removing_sent = set([sent for sent in flatten if ((phrase in sent) or (len(sent.split()) < 4))])

        for sent in removing_sent:
            sentences_to_remove.append(sent)

    if("." in sentences_to_remove):
        while("." in sentences_to_remove):
            sentences_to_remove.remove(".")
    if(" ." in sentences_to_remove):
        while(" ." in sentences_to_remove):
            sentences_to_remove.remove(" .")

#    print(sentences_to_remove)
    sentences_to_remove = list(set(sentences_to_remove))
    sentences_to_remove.sort(key= len, reverse = True)  # sort starting by largest sentence, in case smaller sentence get chosen beforehand

    for sent in sentences_to_remove:
        df.text = df.text.apply(lambda x: str(x).replace(sent, ""))

#     print(sentences_to_remove)
    return df



In [10]:
def cosine_preprocess(df, model, threshold):

    sent_removed = []
    time_taken = []
    new_txts = []
    
    for i in range(len(df)):
        
        idx_remove = []
        orig_txt = df['text'].iloc[i]
        sent_text = nltk.sent_tokenize(orig_txt) #Sentence Segmentation
        docu_embeddings = model.encode(orig_txt) #Document Embedding
        sentence_embeddings_ = model.encode(sent_text) #Sentence Embedding

        for k in range(len(sentence_embeddings_)):
            
            sim = cosine(docu_embeddings, sentence_embeddings_[k])

            if sim > threshold:
                idx_remove.append(k)

        idx_remove.sort()
        idx_remove.reverse()

        for j in idx_remove:
            sent_removed.append(sent_text[j])
            del sent_text[j]

        new_txt = ' '.join(sent_text)
        new_txts.append(new_txt)
        
    df['text'] = new_txts

    return df

## Saving original file, to do comparison later

In [11]:
for keys1 in overall_dict.keys():
    for keys2 in overall_dict[keys1].keys():
        df = overall_dict[keys1][keys2]
        df = text_initial_preproecss(df)

        df.apply(save_file,args=('orig',),axis=1)
        

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
import string
from textblob import TextBlob, Word

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

stopwords.words('english')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_stopwords(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def remove_digits(text):
    text = re.sub(r'\d+','', text)
    return text

def remove_punct(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in string.punctuation]
    return ' '.join(filtered_sentence)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def nltk_lemmatizer(text):
    '''
    lemmatize each word in sentence if the word is not punctuation
    Below version is just for ease of understanding
    '''
    words = word_tokenize(text)
    lem_words = []
    for word in words:
        if word not in string.punctuation:
            lem_words.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))

    return ' '.join(lem_words)

def lemmatize_with_postag(text):
    sent = TextBlob(text)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hojinlee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## N-Gram

In [13]:
cosine_process = False
N_gram_process = True
merged_process = True if cosine_process and N_gram_process else False
cosine_threshold=0.95

time_taken = []
text_list = []

if merged_process:
    base='merged'
elif cosine_process:
    base='cosine'
else:
    base='ngram'

print("Methodology: {}".format(base))
  
for keys1 in overall_dict.keys():
    if keys1 != Country_to_process:
        pass
    else:
        for keys2 in overall_dict[keys1].keys(): # i.e. 'US', "CA"

            a = datetime.datetime.now()

            df = overall_dict[keys1][keys2]
            df = text_initial_preproecss(df)

            b = datetime.datetime.now()

            if merged_process:
                df = ngram_preprocess(df)
                df = cosine_preprocess(df,model,cosine_threshold)

            elif cosine_process:
                df = cosine_preprocess(df,model,cosine_threshold)

            elif N_gram_process:
                df = ngram_preprocess(df)


            df.apply(save_file,args = (base,),axis=1)

            df.text = df.text.apply(remove_stopwords)
            df.text = df.text.apply(remove_punct)
            df.text = df.text.apply(remove_digits)
            df.text = df.text.apply(lemmatize_with_postag)

            #saving files
            df.apply(save_file,args = ('lemmatized',),axis=1)

            c = datetime.datetime.now()

            print("Running time, country: {}, publisher: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys1,keys2, b-a, c-b, c-a))
            time_taken.append(c-a)

print(np.mean(time_taken))
print(np.sum(time_taken))

Methodology: ngram
Running time, country: IN, publisher: News Nation, punct_t: 0:00:00.157136, text_proc_t: 0:00:32.392307, total_t: 0:00:32.549443
Running time, country: IN, publisher: Gulte, punct_t: 0:00:00.080560, text_proc_t: 0:00:04.756247, total_t: 0:00:04.836807
Running time, country: IN, publisher: Moneycontrol.com, punct_t: 0:00:01.128391, text_proc_t: 0:02:38.011705, total_t: 0:02:39.140096
Running time, country: IN, publisher: oneindia.com, punct_t: 0:00:00.065014, text_proc_t: 0:00:05.838386, total_t: 0:00:05.903400
Running time, country: IN, publisher: Businessinsider India, punct_t: 0:00:00.048450, text_proc_t: 0:00:03.964102, total_t: 0:00:04.012552
Running time, country: IN, publisher: Telegraph India, punct_t: 0:00:00.100234, text_proc_t: 0:00:16.385720, total_t: 0:00:16.485954
Running time, country: IN, publisher: Reuters India, punct_t: 0:00:00.637056, text_proc_t: 0:01:50.688348, total_t: 0:01:51.325404
Running time, country: IN, publisher: afaqs, punct_t: 0:00:00.

Running time, country: IN, publisher: thebetterindia.com, punct_t: 0:00:00.088195, text_proc_t: 0:00:12.042301, total_t: 0:00:12.130496
Running time, country: IN, publisher: Forbes India, punct_t: 0:00:00.096277, text_proc_t: 0:00:12.987669, total_t: 0:00:13.083946
Running time, country: IN, publisher: Indian Express, punct_t: 0:00:00.290865, text_proc_t: 0:00:33.499994, total_t: 0:00:33.790859
Running time, country: IN, publisher: Fibre2fashion.com, punct_t: 0:00:00.107338, text_proc_t: 0:00:15.123542, total_t: 0:00:15.230880
Running time, country: IN, publisher: economictimes.indiatimes.com, punct_t: 0:00:00.065262, text_proc_t: 0:00:07.560310, total_t: 0:00:07.625572
Running time, country: IN, publisher: Scroll.in, punct_t: 0:00:00.463942, text_proc_t: 0:02:27.105599, total_t: 0:02:27.569541
Running time, country: IN, publisher: firstpost.com, punct_t: 0:00:00.076818, text_proc_t: 0:00:08.283408, total_t: 0:00:08.360226
Running time, country: IN, publisher: Assam Tribune, punct_t: 0

Running time, country: IN, publisher: indiatoday.in, punct_t: 0:00:00.076764, text_proc_t: 0:00:11.495538, total_t: 0:00:11.572302
Running time, country: IN, publisher: The Asian Age, punct_t: 0:00:00.173444, text_proc_t: 0:00:39.687820, total_t: 0:00:39.861264
Running time, country: IN, publisher: livelaw.in, punct_t: 0:00:00.097721, text_proc_t: 0:00:11.424211, total_t: 0:00:11.521932
Running time, country: IN, publisher: BoldSky, punct_t: 0:00:00.088581, text_proc_t: 0:00:12.872550, total_t: 0:00:12.961131
Running time, country: IN, publisher: Asian Correspondent, punct_t: 0:00:00.119783, text_proc_t: 0:00:17.292549, total_t: 0:00:17.412332
Running time, country: IN, publisher: Bangalore Mirror, punct_t: 0:00:00.056055, text_proc_t: 0:00:05.030144, total_t: 0:00:05.086199
Running time, country: IN, publisher: Sify, punct_t: 0:00:00.077317, text_proc_t: 0:00:08.389216, total_t: 0:00:08.466533
Running time, country: IN, publisher: bgr.in, punct_t: 0:00:00.096770, text_proc_t: 0:00:13.

Running time, country: IN, publisher: newindianexpress.com, punct_t: 0:00:00.086572, text_proc_t: 0:00:12.028800, total_t: 0:00:12.115372
Running time, country: IN, publisher: SportsCafe, punct_t: 0:00:00.027709, text_proc_t: 0:00:00.966913, total_t: 0:00:00.994622
Running time, country: IN, publisher: ETAuto.com, punct_t: 0:00:00.024824, text_proc_t: 0:00:01.316605, total_t: 0:00:01.341429
Running time, country: IN, publisher: VC Circle, punct_t: 0:00:00.131123, text_proc_t: 0:00:25.146091, total_t: 0:00:25.277214
Running time, country: IN, publisher: millenniumpost.in, punct_t: 0:00:00.026422, text_proc_t: 0:00:00.877913, total_t: 0:00:00.904335
Running time, country: IN, publisher: YourStory.com, punct_t: 0:00:00.062594, text_proc_t: 0:00:08.158825, total_t: 0:00:08.221419
Running time, country: IN, publisher: The Echo of India, punct_t: 0:00:00.045188, text_proc_t: 0:00:03.516558, total_t: 0:00:03.561746
Running time, country: IN, publisher: huffingtonpost.in, punct_t: 0:00:00.0983

Running time, country: IN, publisher: ETBrandEquity.com, punct_t: 0:00:00.023344, text_proc_t: 0:00:01.239863, total_t: 0:00:01.263207
Running time, country: IN, publisher: Jagran Post, punct_t: 0:00:00.041641, text_proc_t: 0:00:03.024904, total_t: 0:00:03.066545
Running time, country: IN, publisher: business-standard.com, punct_t: 0:00:00.051967, text_proc_t: 0:00:04.796564, total_t: 0:00:04.848531
Running time, country: IN, publisher: MensXP.com, punct_t: 0:00:00.047707, text_proc_t: 0:00:04.553377, total_t: 0:00:04.601084
Running time, country: IN, publisher: The Statesman, punct_t: 0:00:00.043201, text_proc_t: 0:00:03.454006, total_t: 0:00:03.497207
Running time, country: IN, publisher: Siliconindia.com, punct_t: 0:00:00.053465, text_proc_t: 0:00:05.602958, total_t: 0:00:05.656423
Running time, country: IN, publisher: Digit, punct_t: 0:00:00.030322, text_proc_t: 0:00:01.658209, total_t: 0:00:01.688531
Running time, country: IN, publisher: ETEnergyworld.com, punct_t: 0:00:00.023030,

## Cosine

In [14]:
# cosine_process = True
# N_gram_process = False
# merged_process = True if cosine_process and N_gram_process else False
# cosine_threshold=0.95

# model = SentenceTransformer('sentence-transformers/bert-base-nli-stsb-mean-tokens')

# time_taken = []
# text_list = []

# if merged_process:
#     base='merged'
# elif cosine_process:
#     base='cosine'
# else:
#     base='ngram'

# print("Methodology: {}".format(base))
  
# for keys in overall_dict.keys(): # i.e. 'US', "CA"
    
#     a = datetime.datetime.now()
    
#     df = overall_dict[keys]   
#     df = text_initial_preproecss(df)
    
#     b = datetime.datetime.now()
    
# #     df.apply(save_file_initial,args = (base,),axis=1)

# #     text_list.append(df.iloc[1])
#     if merged_process:
#         df = ngram_preprocess(df)
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif cosine_process:
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif N_gram_process:
#         df = ngram_preprocess(df)

    
#     #saving files
#     df.apply(save_file,args = (base,),axis=1)

#     c = datetime.datetime.now()
    
#     print("Running time, country: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys, b-a, c-b, c-a))
#     time_taken.append(c-a)

# print(np.mean(time_taken))
# print(np.sum(time_taken))

## Merged

In [15]:
# cosine_process = True
# N_gram_process = True
# merged_process = True if cosine_process and N_gram_process else False
# cosine_threshold=0.95

# model = SentenceTransformer('sentence-transformers/bert-base-nli-stsb-mean-tokens')

# time_taken = []
# text_list = []

# if merged_process:
#     base='merged'
# elif cosine_process:
#     base='cosine'
# else:
#     base='ngram'

# print("Methodology: {}".format(base))
  
# for keys in overall_dict.keys(): # i.e. 'US', "CA"
    
#     a = datetime.datetime.now()
    
#     df = overall_dict[keys]   
#     df = text_initial_preproecss(df)
    
#     b = datetime.datetime.now()
    
# #     df.apply(save_file_initial,args = (base,),axis=1)

# #     text_list.append(df.iloc[1])
#     if merged_process:
#         df = ngram_preprocess(df)
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif cosine_process:
#         df = cosine_preprocess(df,model,cosine_threshold)

#     elif N_gram_process:
#         df = ngram_preprocess(df)

    
#     #saving files
#     df.apply(save_file,args = (base,),axis=1)

#     c = datetime.datetime.now()
    
#     print("Running time, country: {}, punct_t: {}, text_proc_t: {}, total_t: {}".format(keys, b-a, c-b, c-a))
#     time_taken.append(c-a)

# print(np.mean(time_taken))
# print(np.sum(time_taken))